Note that this script need to install equilibrator_api

In [1]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt
import cobra
from equilibrator_cache.thermodynamic_constants import *
from equilibrator_api import ComponentContribution, Q_, Reaction

from dGbyG.config import package_path

cc = ComponentContribution()

In [2]:
default_T = 298.15
default_I = 0.25
default_pH = 7.0
default_pMg = 14.0

conditions = {'c':{'pH':7.20, 'e_potential':0, 'T':default_T, 'I':default_I, 'pMg':default_pMg},
              'e':{'pH':7.40, 'e_potential':30 * 1e-3, 'T':default_T, 'I':default_I, 'pMg':default_pMg},
              'n':{'pH':7.20, 'e_potential':0, 'T':default_T, 'I':default_I, 'pMg':default_pMg},
              'r':{'pH':7.20, 'e_potential':0, 'T':default_T, 'I':default_I, 'pMg':default_pMg},
              'g':{'pH':6.35, 'e_potential':0, 'T':default_T, 'I':default_I, 'pMg':default_pMg},
              'l':{'pH':5.50, 'e_potential':19 * 1e-3, 'T':default_T, 'I':default_I, 'pMg':default_pMg},
              'm':{'pH':8.00, 'e_potential':-155 * 1e-3, 'T':default_T, 'I':default_I, 'pMg':default_pMg},
              'i':{'pH':8.00, 'e_potential':-155 * 1e-3, 'T':default_T, 'I':default_I, 'pMg':default_pMg},
              'x':{'pH':7.00, 'e_potential':12 * 1e-3, 'T':default_T, 'I':default_I, 'pMg':default_pMg}}

### 1. Predicting standard Gibbs energy for Recon3D

In [ ]:
recon3d = cobra.io.load_matlab_model(os.path.join(package_path, 'data/Recon3D/Recon3D_301.mat'))
S = cobra.util.array.create_stoichiometric_matrix(recon3d) # shape = [met, rxn]

# patch
recon3d.metabolites.get_by_id('aqcobal[e]').annotation['pubchem.compound'] = ['4238']
recon3d.metabolites.get_by_id('aqcobal[c]').annotation['pubchem.compound'] = ['4238']
recon3d.metabolites.get_by_id('yvite[e]').annotation['kegg.compound'] = ['C02483']

Restricted license - for non-production use only - expires 2025-11-24


No defined compartments in model Recon3D. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [5]:
IDs = set({})
for met in recon3d.metabolites:
    for key, value in met.annotation.items():
        IDs |= set([key])
        assert len(value) == 1
IDs

{'SMILES', 'chebi', 'hmdb', 'inchi', 'kegg.compound', 'pubchem.compound'}

In [6]:
compound_dict = {}
# obtain a list of compound objects using `get_compound`
for met in recon3d.metabolites:
    kegg = met.annotation.get('kegg.compound', [None])[0]
    chebi = met.annotation.get('chebi', [None])[0]
    hmdb = met.annotation.get('hmdb', [None])[0]
    inchi = met.annotation.get('inchi', [None])[0]

    compound = None

    if ((compound is None) or (compound.inchi_key is None)) and kegg:
        compound = cc.get_compound(f"kegg:{kegg}")
        pass
    if ((compound is None) or (compound.inchi_key is None)) and chebi:
        compound = cc.get_compound(f"chebi:{chebi}")
        pass
    if ((compound is None) or (compound.inchi_key is None)) and hmdb:
        compound = cc.get_compound(f"hmdb:{hmdb}")
        pass
    if ((compound is None) or (compound.inchi_key is None)) and inchi:
        compound = cc.get_compound_by_inchi(inchi)
        pass
    if (compound is None) or (compound.inchi_key is None):
        compound = cc.get_compound(f"bigg.metabolite:{met.id[:-3]}")

    if compound is None:
        compound_dict[met.id] = None
        met.compound = None
    elif compound.inchi_key is None:
        compound_dict[met.id] = None
        met.compound = compound
    else:
        compound_dict[met.id] = compound
        met.compound = compound

    met.met = met.id[:-3]
    met.kegg = kegg
    met.chebi = chebi
    met.hmdb = hmdb
    met.inchi = inchi
    
sum([x is not None for x in np.array(list(compound_dict.values()))])

4064

In [23]:
dGf = []
for met in tqdm(recon3d.metabolites):
    if met.compound is None:
        dGf.append(None)
    else:
        dGf.append(cc.standard_dg_formation(met.compound)[0])

dGf = np.array(dGf, dtype=np.float64)
sum(~np.isnan(dGf))

100%|██████████| 8399/8399 [00:00<00:00, 20037.45it/s]


3876

In [29]:
dGf_df = pd.DataFrame(dGf, columns=['standard dGf',], index=[met.id for met in recon3d.metabolites])
dGf_df.to_csv(os.path.join(package_path, 'data/Recon3D/Recon3D_standard_dGf_eQuilibrator.csv'))

In [28]:
dGr = []
for rxn in recon3d.reactions:
    r = {}
    for k, coeff in cc.parse_formula(rxn.reaction).items():
        r[compound_dict[k]] = coeff + r.get(compound_dict[k], 0)
    if None in r:
        dGr.append([np.nan, np.nan])
    else:
        reaction = Reaction(r)
        if reaction.is_balanced() == False:
            dGr.append([np.nan, np.nan])
        else:
            dgr = cc.standard_dg(reaction).m_as("kJ/mol")
            dgr = [dgr.n, dgr.s]
            if all([x.can_be_transformed() for x in r.keys()]) == True:
                for met in rxn.metabolites:
                    cpd = met.compound
                    v = rxn.get_coefficient(met.id)
                    cdt = conditions[met.compartment]
                    dgr[0] += cpd.transform(Q_(cdt['pH']), Q_(cdt['I'], 'M'), Q_(cdt['T'], 'K'), Q_(cdt['pMg'])).m_as("kJ/mol") * v
                dGr.append(dgr)
            else:
                dGr.append([np.nan, np.nan])

dGr = np.array(dGr)
sum(~np.isnan(dGr))

array([5885, 5885])

In [9]:
dGr_df = pd.DataFrame(dGr, columns=['transformed standard dGr', 'SD'])
dGr_df.to_csv(os.path.join(package_path, 'data/Recon3D/Recon3D_standard_dGr_eQuilibrator.csv'))

In [10]:
metNoComp = {}
for met, compound in compound_dict.items():
    if metNoComp.get(met[:-3], None) is None:
        metNoComp[met[:-3]] = None if compound is None else cc.standard_dg_formation(compound)[0]
sum([x is not None for x in metNoComp.values()]), len(metNoComp), \
sum([x is not None for x in metNoComp.values()])/len(metNoComp)

(1615, 4140, 0.39009661835748793)

### 2. Predicting standard Gibbs energy for Human1

In [30]:
# Read model and patch it
human1 = cobra.io.read_sbml_model(os.path.join(package_path, 'data/Human1/Human-GEM/model/Human-GEM.xml'))
human1.metabolites.get_by_id('MAM01935e').annotation['kegg.compound'] = 'C02483'

In [31]:
IDs = set({})
for met in human1.metabolites:
    for key, value in met.annotation.items():
        IDs |= set([key])
IDs

{'bigg.metabolite',
 'chebi',
 'hmdb',
 'inchi',
 'kegg.compound',
 'lipidmaps',
 'metanetx.chemical',
 'pubchem.compound',
 'sbo',
 'vmhmetabolite'}

In [32]:
compound_dict = {}
# obtain a list of compound objects using `get_compound`
for met in human1.metabolites:
    kegg = met.annotation.get('kegg.compound')
    chebi = met.annotation.get('chebi')
    hmdb = met.annotation.get('hmdb')
    bigg = met.annotation.get('bigg.metabolite')
    metanetx = met.annotation.get('metanetx.chemical')
    lipidmaps = met.annotation.get('lipidmaps')
    inchi = met.annotation.get('inchi')
    
    compound = None

    if ((compound is None) or (compound.inchi_key is None)) and kegg:
        compound = cc.get_compound(f"kegg:{kegg}")
        pass
    if ((compound is None) or (compound.inchi_key is None)) and chebi:
        compound = cc.get_compound(f"chebi:{chebi}")
        pass
    if ((compound is None) or (compound.inchi_key is None)) and hmdb:
        compound = cc.get_compound(f"hmdb:{hmdb}")
        pass
    if ((compound is None) or (compound.inchi_key is None)) and bigg:
        compound = cc.get_compound(f"bigg.metabolite:{bigg}")
        pass
    if ((compound is None) or (compound.inchi_key is None)) and metanetx:
        compound = cc.get_compound(f"metanetx.chemical:{metanetx}")
        pass
    if ((compound is None) or (compound.inchi_key is None)) and lipidmaps:
        compound = cc.get_compound(f"lipidmaps:{lipidmaps}")
        pass
    if ((compound is None) or (compound.inchi_key is None)) and inchi:
        compound = cc.get_compound_by_inchi(inchi)
        pass

    if compound is None:
        compound_dict[met.id] = None
        met.compound = None
    elif compound.inchi_key is None:
        compound_dict[met.id] = None
        met.compound = compound
    else:
        compound_dict[met.id] = compound
        met.compound = compound

    met.met = met.id
    met.kegg = kegg
    met.chebi = chebi
    met.hmdb = hmdb
    met.bigg = bigg
    met.metanetx = metanetx
    met.lipidmaps = lipidmaps
    met.inchi = inchi
    
sum([x is not None for x in np.array(list(compound_dict.values()))])

4266

In [33]:
dGf = []
for met in tqdm(human1.metabolites):
    if met.compound is None:
        dGf.append(None)
    else:
        dGf.append(cc.standard_dg_formation(met.compound)[0])

dGf = np.array(dGf, dtype=np.float64)
sum(~np.isnan(dGf))

100%|██████████| 8456/8456 [00:01<00:00, 5808.36it/s] 


4078

In [34]:
dGf_df = pd.DataFrame(dGf, columns=['standard dGf',], index=[met.id for met in human1.metabolites])
dGf_df.to_csv(os.path.join(package_path, 'data/Human1/Human1_standard_dGf_eQuilibrator.csv'))

In [22]:
dGr = []
for rxn in human1.reactions:
    r = {}
    for k, coeff in cc.parse_formula(rxn.reaction).items():
        r[compound_dict[k]] = coeff + r.get(compound_dict[k], 0)
    if None in r:
        dGr.append([np.nan, np.nan])
    else:
        reaction = Reaction(r)
        if reaction.is_balanced() == False:
            dGr.append([np.nan, np.nan])
        else:
            dgr = cc.standard_dg(reaction).m_as("kJ/mol")
            dgr = [dgr.n, dgr.s]
            if all([x.can_be_transformed() for x in r.keys()]) == True:
                for met in rxn.metabolites:
                    cpd = met.compound
                    v = rxn.get_coefficient(met.id)
                    cdt = conditions[met.compartment]
                    dgr[0] += cpd.transform(Q_(cdt['pH']), Q_(cdt['I'], 'M'), Q_(cdt['T'], 'K'), Q_(cdt['pMg'])).m_as("kJ/mol") * v
                dGr.append(dgr)
            else:
                dGr.append([np.nan, np.nan])

dGr = np.array(dGr)
sum(~np.isnan(dGr))

array([6013, 6013])

In [23]:
dGr_df = pd.DataFrame(dGr, columns=['transformed standard dGr', 'SD'])
dGr_df.to_csv(os.path.join(package_path, 'data/Human1/Human1_standard_dGr_eQuilibrator.csv'))

In [28]:
metNoComp = {}
for met, compound in compound_dict.items():
    if metNoComp.get(met[:-1], None) is None:
        metNoComp[met[:-1]] = None if compound is None else cc.standard_dg_formation(compound)[0]
sum([x is not None for x in metNoComp.values()]), len(metNoComp), \
sum([x is not None for x in metNoComp.values()])/len(metNoComp)

(1766, 4156, 0.42492781520692974)